## DeepLearning Model

#### Import libraries

In [29]:
import joblib
import pandas as pd
import sklearn
#import sklearn.datasets
from keras.models import Sequential
from keras.layers import Dense
import warnings

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)

tr_features = pd.read_csv('../Resources/train_features.csv')
tr_labels = pd.read_csv('../Resources/train_labels.csv', header=None)
test_features = pd.read_csv('../Resources/test_features.csv')
test_labels = pd.read_csv('../Resources/test_labels.csv', header=None)

val_features = pd.read_csv('../Resources/val_features.csv')
val_labels = pd.read_csv('../Resources/val_labels.csv', header=None)

In [30]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [31]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(tr_features)
X_train_scaled = X_scaler.transform(tr_features)
X_test_scaled = X_scaler.transform(test_features)
X_val_scaled = X_scaler.transform(val_features)

#### Train Model

In [70]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [71]:
#sparse_categorical_crossentropy was used because of multiple levels of the y-value (0, 1, 2)
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 100)               4100      
_________________________________________________________________
dense_25 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_26 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 153       
Total params: 11,853
Trainable params: 11,853
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.fit(
X_train_scaled,
tr_labels.values,
epochs=60,
shuffle=True,
verbose=2
)

Epoch 1/60
 - 1s - loss: 0.5689 - accuracy: 0.7213
Epoch 2/60
 - 0s - loss: 0.3721 - accuracy: 0.8035
Epoch 3/60
 - 0s - loss: 0.3591 - accuracy: 0.8164
Epoch 4/60
 - 0s - loss: 0.3487 - accuracy: 0.8267
Epoch 5/60
 - 0s - loss: 0.3384 - accuracy: 0.8269
Epoch 6/60
 - 0s - loss: 0.3286 - accuracy: 0.8410
Epoch 7/60
 - 0s - loss: 0.3222 - accuracy: 0.8443
Epoch 8/60
 - 0s - loss: 0.3208 - accuracy: 0.8495
Epoch 9/60
 - 0s - loss: 0.3094 - accuracy: 0.8584
Epoch 10/60
 - 0s - loss: 0.3189 - accuracy: 0.8484
Epoch 11/60
 - 0s - loss: 0.3137 - accuracy: 0.8517
Epoch 12/60
 - 0s - loss: 0.3027 - accuracy: 0.8591
Epoch 13/60
 - 0s - loss: 0.2937 - accuracy: 0.8698
Epoch 14/60
 - 0s - loss: 0.3009 - accuracy: 0.8591
Epoch 15/60
 - 0s - loss: 0.2923 - accuracy: 0.8691
Epoch 16/60
 - 0s - loss: 0.2871 - accuracy: 0.8751
Epoch 17/60
 - 0s - loss: 0.2932 - accuracy: 0.8617
Epoch 18/60
 - 0s - loss: 0.2938 - accuracy: 0.8629
Epoch 19/60
 - 0s - loss: 0.2818 - accuracy: 0.8770
Epoch 20/60
 - 0s - l

#### Model evaluation and Prediction on test-data

In [73]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, test_labels, verbose=2)
print(
    f"result_df Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

result_df Neural Network - Loss: 0.25116429563672993, Accuracy: 0.8970693349838257


In [74]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
Actual_label = (test_labels[:20]).values.ravel()

In [75]:
result_df = pd.DataFrame({
    'Prediction': encoded_predictions,
    'Actual': Actual_label
})
result_df

,Prediction,Actual
0,2,2
1,0,0
2,2,2
3,0,0
4,0,1
5,2,2
6,1,1
7,0,0
8,2,2
9,2,2


#### Write Model

In [76]:
joblib.dump(model, '../Saved_model/DL_model.pkl')

['../Saved_model/DL_model.pkl']

#### Model validation using validation-data

In [77]:
 model_val = joblib.load('../Saved_model/DL_model.pkl')

In [78]:
model_loss, model_accuracy = model_val.evaluate(
    X_val_scaled, val_labels, verbose=2)
print(
    f"result_df Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

result_df Neural Network - Loss: 0.2624242850639278, Accuracy: 0.8977110385894775
